# 🔍 Laboratório: Busca Semântica Local com Ollama

## O que vamos aprender?

Neste notebook, você vai construir um **sistema de busca inteligente** que entende o **significado** das palavras, não apenas correspondências exatas de texto.

### Diferença entre Busca Tradicional vs. Busca Semântica

**Busca Tradicional (palavras-chave):**
```text
Query: "hardware celular"
Resultado: Só encontra documentos com as palavras exatas "hardware" e "celular"
```

**Busca Semântica (significado):**
```text
Query: "hardware celular"
Resultado: Encontra "iPhone", "smartphone", "processador mobile" 
           mesmo sem as palavras exatas!
```

### Como isso funciona?

Usando **bibliotecas reais** de produção que empresas usam no dia a dia:
- **LangChain** → Framework para construir apps com IA
- **Ollama** → Roda modelos de IA localmente (grátis!)
- **FAISS** → Banco de dados vetorial ultrarrápido

## 🛠️ Ferramentas que Vamos Usar

### 1. LangChain 🦜🔗
**O que é:** Framework Python que simplifica a criação de aplicações com IA  
**Por que usar:** Em vez de escrever centenas de linhas de código, usamos algumas linhas  
**Analogia:** É como usar um framework web (Django/Flask) em vez de programar HTTP do zero

### 2. Ollama 🦙
**O que é:** Plataforma para rodar modelos de IA **localmente** no seu computador  
**Por que usar:** 
- ✅ Gratuito (sem pagar por API)
- ✅ Privado (seus dados não vão para a nuvem)
- ✅ Rápido (sem latência de internet)

**Modelos de embedding disponíveis:**
- `all-minilm`: 384 dimensões, rápido e leve
- `nomic-embed-text`: 768 dimensões, balanceado
- `mxbai-embed-large`: 1024 dimensões, mais preciso

### 3. FAISS 🚀
**O que é:** Biblioteca do Facebook/Meta para busca vetorial eficiente  
**Por que usar:** Consegue buscar em **milhões de vetores** em milissegundos  
**Analogia:** É como um índice de banco de dados, mas para vetores matemáticos

**Como funciona:**
```text
Texto → [0.2, -0.5, 0.8, ...] ← Vetor com 384/768/1024 números
         ↓
      Armazenado no FAISS
         ↓
Query → [0.3, -0.4, 0.7, ...] ← Também vira vetor
         ↓
      FAISS compara distâncias
         ↓
    Retorna os mais próximos!
```

## 🎯 Cenário do Experimento

Vamos criar um **"mini Google"** com documentos de 3 categorias completamente diferentes:

| Categoria | Documentos |
|-----------|------------|
| 🖥️ **Tecnologia** | iPhone 15, RTX 4090 |
| 🍰 **Culinária** | Receita de bolo, Lasanha |
| ⚽ **Esportes** | Gol de futebol |

### O Desafio

Faremos uma pergunta que **não contém palavras exatas** dos documentos:

**Pergunta:** "Quero sugestões de hardware para computador ou celular"

**Expectativa:**
- ✅ Deve retornar: iPhone e RTX 4090 (são hardware!)
- ❌ NÃO deve retornar: Bolo, Lasanha, Gol (não são hardware)

### Por que isso é difícil?

Um sistema de busca tradicional (Ctrl+F) falharia porque:
- A palavra "iPhone" não aparece na pergunta
- A palavra "RTX 4090" não aparece na pergunta
- Mas nossa IA precisa **entender** que ambos são hardware!

💡 **Isso é Inteligência Artificial em ação!** O modelo aprendeu que "iPhone = celular = hardware" e "RTX 4090 = placa de vídeo = hardware" durante seu treinamento.

In [1]:
%pip install faiss-cpu>=1.13.1 langchain-openai==1.1.0  langchain-ollama==1.0.0 ollama==0.6.1 langchain-community>=0.4.1

Note: you may need to restart the kernel to use updated packages.


## 📦 Instalação de Dependências

Antes de começar, precisamos instalar as bibliotecas necessárias:

- `faiss-cpu`: Busca vetorial (versão CPU)
- `langchain-ollama`: Integração Ollama + LangChain
- `langchain-community`: Utilitários do LangChain
- `ollama`: Cliente Python do Ollama

⚠️ **Nota:** Se você já instalou essas bibliotecas, pode pular esta célula.

In [2]:
import os
import requests
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.vectorstores import FAISS

# Se preferir usar Ollama Embeddings
from langchain_ollama import OllamaEmbeddings


## 🔌 Imports e Configuração Inicial

Importando as bibliotecas que vamos usar:

- `os`: Para ler variáveis de ambiente
- `requests`: Para verificar se o Ollama está rodando
- `Path` e `load_dotenv`: Para carregar configurações do arquivo `.env`
- `FAISS`: O banco de dados vetorial
- `OllamaEmbeddings`: O modelo que transforma texto em vetores

In [3]:
# 2) Configuração e carregamento do .env (simplificado)
env_path = Path.cwd().joinpath('..', '..', '.env').resolve()
if env_path.exists():
    load_dotenv(env_path)
    print(f'🔎 .env carregado -> {env_path.resolve()}')
else:
    print('⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.')


⚠️  .env não encontrado. Defina as variáveis de ambiente manualmente.


## 🔐 Carregando Variáveis de Ambiente

O arquivo `.env` contém configurações como:
- URL do Ollama (`OLLAMA_API_URL`)
- Chaves de API (se necessário)

**Por que usar `.env`?**
- ✅ Não expor senhas no código
- ✅ Fácil mudar entre ambientes (dev/prod)
- ✅ Compartilhar código sem vazar credenciais

In [4]:
# Configuração do Ollama (ajuste conforme necessário)
# - Dentro do Docker: http://ollama:11434
# - Fora do Docker: http://localhost:11434
OLLAMA_API_URL = os.getenv('OLLAMA_API_URL', 'http://localhost:11434')

# Verificar se Ollama está online
def check_ollama_health() -> bool:
    try:
        response = requests.get(f'{OLLAMA_API_URL}/api/tags', timeout=5)
        if response.status_code == 200:
            models = response.json().get('models', [])
            print(f'✅ Ollama está online! Modelos disponíveis: {len(models)}')
            for model in models:
                name = model.get('name', 'unknown')
                size = model.get('size', 0) / (1024**3)  # Convert to GB
                print(f'   - {name} ({size:.2f} GB)')
            return True
        else:
            print(f'❌ Ollama retornou status {response.status_code}')
            return False
    except Exception as e:
        print(f'❌ Erro ao conectar com Ollama: {e}')
        return False

# Testar conexão
check_ollama_health()

✅ Ollama está online! Modelos disponíveis: 3
   - all-minilm:latest (0.04 GB)
   - mxbai-embed-large:latest (0.62 GB)
   - nomic-embed-text:latest (0.26 GB)


True

## 🏥 Health Check do Ollama

Antes de usar o Ollama, precisamos verificar se ele está rodando corretamente.

**O que esta célula faz:**
1. Tenta conectar na URL do Ollama (padrão: `http://localhost:11434`)
2. Lista todos os modelos disponíveis
3. Mostra o tamanho de cada modelo em GB

**Possíveis URLs:**
- `http://localhost:11434` → Se Ollama está rodando no seu computador
- `http://ollama:11434` → Se Ollama está rodando em Docker Compose

💡 **Dica:** Se você ver ❌, o Ollama não está rodando. Inicie-o com `ollama serve` no terminal.

## 📄 Passo 1: Preparando os Documentos

Aqui temos nosso **dataset de teste** — 5 documentos de categorias bem distintas.

**Por que categorias diferentes?**
Queremos testar se a IA consegue distinguir contextos. Um bom modelo de embedding deve:
- Colocar "iPhone" e "RTX 4090" próximos (ambos são tecnologia/hardware)
- Colocar "bolo" e "lasanha" próximos (ambos são culinária)
- Manter "gol" distante de receitas e hardware (esporte é outro contexto)

### Estrutura dos Dados

```python
meus_textos = [
    "Documento sobre tecnologia...",  # Categoria: Tech
    "Documento sobre culinária...",   # Categoria: Food
    "Documento sobre esporte...",     # Categoria: Sport
]
```

💡 **Conceito importante:** Esses textos são chamados de **corpus** (conjunto de documentos que queremos buscar).

In [5]:

meus_textos = [
    "O novo iPhone 15 tem uma lente periscópica incrível.",    # Tecnologia
    "Para fazer um bolo macio, bata as claras em neve.",       # Culinária
    "O atacante chutou a bola no ângulo e foi gol.",           # Esporte
    "A placa de vídeo RTX 4090 roda jogos em 4K.",             # Tecnologia
    "Receita de lasanha à bolonhesa com muito queijo."         # Culinária
]

## 🧠 Passo 2: Inicializando o Modelo de Embeddings

Este é o **"cérebro"** que transforma texto em vetores numéricos!

### O que são Embeddings?

**Embedding** = Representação numérica de um texto

Exemplo:
```text
Texto: "gato"
Embedding: [0.12, -0.34, 0.56, 0.89, ..., 0.23]
           ↑ vetor com 384 números (all-minilm)
```

### Como funciona a magia?

1. O modelo foi **treinado** em milhões de textos
2. Aprendeu que "gato" e "felino" têm significados parecidos
3. Então gera vetores **próximos** para palavras com significados similares

**Visualização conceitual:**
```text
Espaço vetorial (imagine em 3D, mas é 384D!)

  gato • ← próximo de → • felino
  
  carro • ← DISTANTE de → • felino
```

### Modelos Disponíveis

| Modelo | Dimensões | Velocidade | Qualidade |
|--------|-----------|------------|-----------|
| `all-minilm` | 384 | ⚡⚡⚡ | ⭐⭐ |
| `nomic-embed-text` | 768 | ⚡⚡ | ⭐⭐⭐ |
| `mxbai-embed-large` | 1024 | ⚡ | ⭐⭐⭐⭐ |

**Qual escolher?**
- Teste rápido/protótipo: `all-minilm`
- Produção balanceada: `nomic-embed-text`
- Máxima precisão: `mxbai-embed-large`

💡 **Nota:** Todos esses modelos rodam **localmente** e são **gratuitos**!

In [6]:
# Se preferir usar Ollama Embeddings
embeddings = OllamaEmbeddings(
    # model="mxbai-embed-large",
    # model="nomic-embed-text",
    model="all-minilm",
    base_url=OLLAMA_API_URL,
    # Em vez de headers, use client kwargs (se necessário para auth)
    client_kwargs={"headers": {"Authorization": "Bearer YOUR_TOKEN"}} if os.getenv("OLLAMA_API_KEY") else {},
    # opcional: validar se o modelo existe localmente
    validate_model_on_init=False,
    # opcional: tempo de keep-alive
    keep_alive=5 * 60,
)

## 🗄️ Passo 3: Criando o Banco de Dados Vetorial (Indexação)

Aqui acontece a **mágica**! Vamos transformar todos os textos em vetores e armazená-los no FAISS.

### O que acontece nesta linha de código?

```python
vector_store = FAISS.from_texts(meus_textos, embeddings)
```

**Passo a passo interno:**

1. **Para cada texto** em `meus_textos`:
   ```text
   "O novo iPhone 15..." → embeddings.embed_query() → [0.2, -0.5, 0.8, ...]
   "Para fazer um bolo..." → embeddings.embed_query() → [0.1, 0.3, -0.4, ...]
   ...
   ```

2. **FAISS armazena** todos esses vetores em uma estrutura otimizada:
   ```text
   Índice FAISS:
   [0] → [0.2, -0.5, 0.8, ...] (iPhone)
   [1] → [0.1, 0.3, -0.4, ...] (bolo)
   [2] → [-0.3, 0.7, 0.2, ...] (gol)
   [3] → [0.25, -0.48, 0.82, ...] (RTX 4090)
   [4] → [0.12, 0.28, -0.38, ...] (lasanha)
   ```

3. **Pronto!** Agora podemos fazer buscas ultrarrápidas

### Por que FAISS é rápido?

FAISS usa algoritmos avançados como:
- **IVF** (Inverted File Index): Agrupa vetores similares
- **HNSW** (Hierarchical Navigable Small World): Grafo para busca eficiente

**Resultado:** Consegue buscar em milhões de vetores em milissegundos! ⚡

### Analogia

Imagine que você tem uma biblioteca com milhões de livros:
- **Sem índice:** Você precisa ler cada livro para achar o que quer 😰
- **Com FAISS:** É como ter um sistema Dewey Decimal que te leva direto ao livro certo! 📚✨

In [7]:
vector_store = FAISS.from_texts(meus_textos, embeddings)

---

## 🚀 Hora de Testar! Vamos Fazer Buscas

Configuração concluída! ✅  
Agora vamos usar nosso sistema de busca semântica.

**O que fizemos até agora:**
1. ✅ Instalamos as bibliotecas
2. ✅ Conectamos ao Ollama
3. ✅ Criamos um modelo de embeddings
4. ✅ Indexamos nossos documentos no FAISS

**Próximo passo:** Fazer perguntas e ver a IA encontrar as respostas! 🎯

## 🔍 Passo 4: Definindo a Pergunta (Query)

Aqui está o **teste de fogo** para nosso sistema!

### A Pergunta

```python
"Quero sugestões de hardware para computador ou celular"
```

### Por que esta pergunta é desafiadora?

**Palavras que NÃO aparecem na pergunta:**
- ❌ "iPhone"
- ❌ "RTX"
- ❌ "4090"
- ❌ "placa de vídeo"
- ❌ "lente periscópica"

**Palavras que SIM aparecem:**
- ✅ "hardware"
- ✅ "computador"
- ✅ "celular"

### O Desafio

Um sistema de busca tradicional (Ctrl+F) **falharia** porque:
```text
Busca por "hardware" → Não encontra nada (palavra não está nos docs)
Busca por "celular" → Não encontra "iPhone" (palavra diferente)
Busca por "computador" → Não encontra "RTX 4090" (palavra diferente)
```

### A Inteligência Artificial

Nossa IA vai **entender** que:
```text
"celular" ≈ "iPhone" ≈ "smartphone"
"computador" ≈ "PC" ≈ "placa de vídeo"
"hardware" ≈ "equipamento" ≈ "dispositivo"
```

💡 **Isso é aprendizado semântico!** O modelo foi treinado para entender relações entre conceitos, não apenas correspondência exata de strings.

In [8]:
pergunta = "Quero sugestões de hardware para computador ou celular"

## 🎯 Passo 5: Executando a Busca Semântica

Agora vamos **executar a busca** e ver os resultados!

### O que significa `k=2`?

```python
vector_store.similarity_search(pergunta, k=2)
```

O parâmetro `k` define quantos resultados queremos:
- `k=1` → Retorna apenas o documento mais similar
- `k=2` → Retorna os 2 documentos mais similares
- `k=5` → Retorna os 5 documentos mais similares

### Como funciona internamente?

1. **Sua pergunta vira um vetor:**
   ```text
   "Quero sugestões de hardware..." → embeddings.embed_query() 
   → [0.22, -0.48, 0.79, ...]
   ```

2. **FAISS calcula distâncias:**
   ```text
   Distância(query, iPhone) = 0.85
   Distância(query, bolo) = 2.34
   Distância(query, gol) = 2.67
   Distância(query, RTX 4090) = 0.92
   Distância(query, lasanha) = 2.41
   ```

3. **Ordena por proximidade:**
   ```text
   1º lugar: iPhone (0.85) ← Mais próximo!
   2º lugar: RTX 4090 (0.92)
   3º lugar: lasanha (2.41)
   ...
   ```

4. **Retorna os top-k:**
   ```text
   Como k=2, retorna: [iPhone, RTX 4090]
   ```

### Métrica de Similaridade

FAISS usa **similaridade cosseno** por padrão:
- Mede o **ângulo** entre dois vetores
- Quanto menor o ângulo, mais similares são os textos
- Funciona bem mesmo com vetores de dimensões diferentes

**Visualização conceitual (2D, mas é 384D!):**

![Distância Euclidiana](distancia_euclidiana.png)

### O que esperar?

✅ **Esperado:** iPhone e RTX 4090 (hardware!)  
❌ **Não esperado:** Bolo, Lasanha, Gol (não são hardware)

Vamos ver se a IA acerta! 👇

In [9]:
resultados = vector_store.similarity_search(pergunta, k=2)

print(f"Pergunta: '{pergunta}'\n")
print("--- Documentos Encontrados ---")
for i, doc in enumerate(resultados):
    print(f"{i+1}. {doc.page_content}")

Pergunta: 'Quero sugestões de hardware para computador ou celular'

--- Documentos Encontrados ---
1. A placa de vídeo RTX 4090 roda jogos em 4K.
2. Receita de lasanha à bolonhesa com muito queijo.


## 📊 Análise dos Resultados

### O que observar?

Quando você executar a célula acima, observe:

1. **Os documentos retornados são relevantes?**
   - ✅ Se retornou iPhone e RTX 4090: O modelo entendeu!
   - ❌ Se retornou receitas ou gol: O modelo falhou

2. **A ordem faz sentido?**
   - O primeiro resultado deve ser o mais relevante
   - O segundo resultado deve ser um pouco menos relevante

### Por que pode falhar?

Modelos diferentes podem interpretar de formas diferentes:
- `all-minilm` é mais rápido, mas menos preciso
- `nomic-embed-text` pode ter viés de treinamento
- `mxbai-embed-large` geralmente é mais confiável



### 🧪 Experimentos para Tentar

#### Experimento 1: Mudar o modelo
```python
# Tente trocar na célula de inicialização:
model="nomic-embed-text"  # ou "mxbai-embed-large"

# E recriar o vector_store:
vector_store = FAISS.from_texts(meus_textos, embeddings)

# E refazer a busca
```



#### Experimento 2: Testar outras queries
```python
# Query específica
pergunta = "receitas de massas italianas"
# Deve retornar: lasanha

# Query ambígua
pergunta = "como melhorar performance"
# Vai retornar hardware ou esporte? 🤔

# Query fora do domínio
pergunta = "viagens para a Europa"
# Vai retornar o que está "menos distante" (mas nada relevante)
```



#### Experimento 3: Ajustar k
```python
# Ver mais resultados
resultados = vector_store.similarity_search(pergunta, k=5)
# Agora você vê TODOS os 5 documentos ordenados por relevância
```



#### Experimento 4: Ver os scores
```python
# Usar similarity_search_with_score para ver as distâncias
resultados = vector_store.similarity_search_with_score(pergunta, k=2)
for doc, score in resultados:
    print(f"Score: {score:.4f} | Texto: {doc.page_content}")

# Score menor = mais similar!
```



### 💡 Conceitos-chave para lembrar

1. **Embedding transforma texto em vetor** (lista de números)
2. **FAISS armazena vetores** e faz buscas rápidas
3. **Similaridade = proximidade no espaço vetorial**
4. **Modelos diferentes = interpretações diferentes**
5. **Sempre teste com dados do SEU domínio!**